In [29]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.special import expit

data = pd.read_csv('神经网络结果.csv')
x1 = data['预测为好客户的得分'].values
x2 = data['预测为坏客户的得分'].values

# 定义违约率可行域
default_rate_thresholds = {
    1: 0.2,
    2: 0.25,
    3: 0.3,
    4: 0.35,
    5: 0.4,
    6: 0.45,
    7: 0.55,
    8: 0.65,
    9: 0.75
}

# 定义目标函数
def objective(params, x1, x2, actual_default_rate):
    a, b = params
    predicted_default_rate = expit(a * x1 + b * x2)  # 使用 Sigmoid 函数预测违约率
    return np.sum((predicted_default_rate - actual_default_rate) ** 2)

# 定义约束条件
def constraint(params, x1, x2, i, actual_default_rate):
    a, b = params
    predicted_default_rate = expit(a * x1 + b * x2)
    max_default_rate = default_rate_thresholds[i]
    return predicted_default_rate[actual_default_rate == 1].mean() - max_default_rate


params_init = np.array([0, 0])

levels = np.arange(1, 10)

constraints = [{'type': 'ineq', 'fun': constraint, 'args': (x1, x2, i, data['Y'].values)} for i in levels]

# SLSQP优化算法求解
result = minimize(objective, params_init, args=(x1, x2, data['Y'].values), constraints=constraints, method='SLSQP')

# 输出结果
if result.success:
    print("最优参数值:", result.x)
    print("目标函数值:", result.fun)
    
    # 更新信用等级并四舍五入为整数
    data['信用等级'] = np.round(expit(result.x[0] * x1 + result.x[1] * x2) * 9 + 1).astype(int)
    
    # 保存数据到 CSV 文件
    data.to_csv('第四问.csv', index=False)
else:
    print("优化失败:", result.message)


最优参数值: [-2.80436038  8.6257203 ]
目标函数值: 278.72606152552
